# Ornstein-Uhlenbeck Process Simulation

The Ornstein-Uhlenbeck process can be descritized using the Euler method from continuous differential equations. The stochastic version of it is called</br>
[Euler-Maruyama](https://en.wikipedia.org/wiki/Euler%E2%80%93Maruyama_method) method. Recal the Ornstein-Uhlenbeck SDE is given by,

$
\begin{align}
dX_t = \lambda \left( \mu^* - X_t \right) dt + \sigma dB_t
\end{align}
$

where $dB_t \sim \text{Normal}(0,dt)$. Replacing the continuous differential with finite differences gives,

$
\begin{align}
&\Delta X_t = \lambda \left( \mu^* - X_t \right) \Delta t + \sigma \Delta B_t \\
\Rightarrow &X_{t+1} = X_t + \lambda \left( \mu^* - X_t \right) \Delta t + \sigma \Delta B_t
\end{align}
$

but $\Delta B_t \sim \text{Normal}(0, \Delta t)$ so,

$
\begin{align}
X_{t+1} = X_t + \lambda \left( \mu^* - X_t \right) \Delta t + \sigma \Delta t \varepsilon_t
\end{align}
$

where $\varepsilon \sim \text{Normal}(0,1)$.



## Imports